Idea is to forecast 1-6 hours til the 48 hours with lookback from 7 days to 30 days.

In [2]:
# =========================
# PatchTST forecasting pipeline (Darts)
# =========================
# If running in a notebook, run this cell once.
# !pip -q install "u8darts[torch]>=0.30.0" torch pytorch-lightning --upgrade

import os
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from pathlib import Path
from datetime import timedelta

# Darts core
from darts import TimeSeries
from darts.models import PatchTSTModel
from darts.dataprocessing.transformers import Scaler
from darts.metrics import mape, rmse, mae

# -----------------
# CONFIG
# -----------------
DATA_CSV = "data/combined_hourly_dataset.csv"   # path to your merged dataset
TARGET_COL = "close"                             # what we forecast
LOOKBACK_H = 168                                 # 7 days of history
HORIZON_H  = 24                                  # forecast next 24 hours
VAL_HOURS  = 24 * 30                             # last 30 days for validation
N_EPOCHS   = 50                                  # training epochs (raise to 100+ for better accuracy)
BATCH_SIZE = 64
RANDOM_SEED = 42
OUT_DIR = Path("data/forecasts")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# -----------------
# 1) Load & preprocess
# -----------------
df = pd.read_csv(DATA_CSV, parse_dates=["timestamp"])
df = df.set_index("timestamp").sort_index()
# keep hourly freq label (use lowercase 'h' to avoid FutureWarning)
df = df.asfreq("h")

# Drop clearly non-numeric columns (e.g., symbol/exchange) and use numeric features only
non_numeric = [c for c in df.columns if not np.issubdtype(df[c].dtype, np.number)]
df_num = df.drop(columns=non_numeric)

# Basic hygiene: forward-fill tiny gaps, then still-missing -> drop rows at the very start if needed
df_num = df_num.ffill().bfill()

# Target & feature split
assert TARGET_COL in df_num.columns, f"{TARGET_COL=} not in columns!"
feature_cols = [c for c in df_num.columns if c != TARGET_COL]
print(f"Using {len(feature_cols)} past covariate features.")

# -----------------
# 2) Build Darts TimeSeries
# -----------------
series_all = TimeSeries.from_dataframe(df_num, value_cols=[TARGET_COL])
past_covs_all = TimeSeries.from_dataframe(df_num[feature_cols]) if feature_cols else None

# Train/val split by time (no leakage)
split_time = series_all.end_time() - pd.Timedelta(hours=VAL_HOURS)
series_train, series_val = series_all.split_after(split_time)
if past_covs_all is not None:
    past_covs_train, past_covs_val = past_covs_all.split_after(split_time)
else:
    past_covs_train = past_covs_val = None

# -----------------
# 3) Scaling (fit on train only)
# -----------------
y_scaler = Scaler()
series_train_scaled = y_scaler.fit_transform(series_train)
series_val_scaled   = y_scaler.transform(series_val)
series_all_scaled   = y_scaler.transform(series_all)

if past_covs_all is not None:
    x_scaler = Scaler()
    past_covs_train_scaled = x_scaler.fit_transform(past_covs_train)
    past_covs_val_scaled   = x_scaler.transform(past_covs_val)
    past_covs_all_scaled   = x_scaler.transform(past_covs_all)
else:
    past_covs_train_scaled = past_covs_val_scaled = past_covs_all_scaled = None

# -----------------
# 4) Define PatchTST
# -----------------
def pick_device():
    import torch
    if torch.cuda.is_available():
        return "cuda"
    try:
        import torch.backends.mps as mps
        if mps.is_available():
            return "mps"
    except Exception:
        pass
    return "cpu"

device = pick_device()
print(f"Using torch device: {device}")

model = PatchTSTModel(
    input_chunk_length=LOOKBACK_H,
    output_chunk_length=HORIZON_H,
    n_epochs=N_EPOCHS,
    batch_size=BATCH_SIZE,
    random_state=RANDOM_SEED,
    d_model=64,             # model width
    n_heads=4,
    num_layers=3,
    dropout=0.1,
    kernel_size=25,         # as in PatchTST paper
    optimizer_kwargs={"lr": 1e-3},
    torch_device=device,
    pl_trainer_kwargs={
        "accelerator": "gpu" if device == "cuda" else ("mps" if device == "mps" else "cpu"),
        "devices": 1,
        "logger": False,
        "enable_checkpointing": True,
    },
)

# -----------------
# 5) Fit
# -----------------
model.fit(
    series=series_train_scaled,
    past_covariates=past_covs_train_scaled,
    val_series=series_val_scaled,
    val_past_covariates=past_covs_val_scaled,
    verbose=True,
)

# -----------------
# 6) Rolling backtest (walk-forward)
# -----------------
# Start backtesting after we have enough history for the lookback
backtest_start = max(series_train_scaled.start_time() + pd.Timedelta(hours=LOOKBACK_H),
                     series_train_scaled.end_time() - pd.Timedelta(hours=VAL_HOURS))

hist_fc = model.historical_forecasts(
    series=series_all_scaled,
    past_covariates=past_covs_all_scaled,
    start=backtest_start,
    forecast_horizon=HORIZON_H,
    stride=HORIZON_H,     # non-overlapping blocks for clarity (set to 1 for tighter evaluation)
    retrain=False,        # set True for strict backtesting (slower)
    verbose=True,
)

# Align & inverse transform forecasts for metrics
hist_fc = TimeSeries.from_series(hist_fc.pd_series()) if isinstance(hist_fc, TimeSeries) else hist_fc
hist_fc_y = y_scaler.inverse_transform(hist_fc)
actual_y  = series_all.slice_intersect(hist_fc_y.time_index)
actual_y  = y_scaler.inverse_transform(actual_y)

print("\n--- Backtest metrics (validation region) ---")
print(f"MAE : {mae(actual_y, hist_fc_y):.4f}")
print(f"RMSE: {rmse(actual_y, hist_fc_y):.4f}")
print(f"MAPE: {mape(actual_y, hist_fc_y):.4f}%")

# -----------------
# 7) Forecast the next HORIZON_H hours
# -----------------
future_fc_scaled = model.predict(
    n=HORIZON_H,
    series=series_all_scaled,
    past_covariates=past_covs_all_scaled,
)

future_fc = y_scaler.inverse_transform(future_fc_scaled)

# Save forecast
pred_df = future_fc.pd_series().to_frame(name=f"predict_{TARGET_COL}")
pred_df.index.name = "timestamp"
pred_path = OUT_DIR / f"patchtst_forecast_{HORIZON_H}h.csv"
pred_df.to_csv(pred_path)
print(f"\nSaved next {HORIZON_H}h forecast → {pred_path}")

# Optional: also dump last train/val portion + prediction for quick plotting elsewhere
tail_df = df_num[[TARGET_COL]].iloc[-(LOOKBACK_H + HORIZON_H + 200):].copy()
tail_df["prediction"] = np.nan
tail_df.loc[pred_df.index, "prediction"] = pred_df[f"predict_{TARGET_COL}"]
tail_dump = OUT_DIR / "patchtst_tail_with_forecast.csv"
tail_df.to_csv(tail_dump)
print(f"Saved tail+forecast slice → {tail_dump}")

ImportError: cannot import name 'PatchTSTModel' from 'darts.models' (/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/darts/models/__init__.py)